In [16]:
import os
import requests
import json
import openai
import openpyxl
import pandas as pd
import spacy
from spacy.lang.en import English

openai.api_key = "YOUR API KEY"
openai.api_base =  "https://YOUR_RESOURCE_NAME.openai.azure.com/" # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2022-12-01' # this may change in the future
deployment_name='YOUR DEPLOYMENT NAME' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

# Send a completion call to generate an answer
print('Sending a test completion job')
start_phrase = 'how are u \n'
response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=10)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase+text) 

Sending a test completion job
how are u 
I'm doing great, thanks for asking.


In [17]:
# read csv
df = pd.read_csv(r'E:\Analysis With Azure Open AI - Chat GPT\input.csv')
print(df)

                                          Review Title  \
0         Another Midrange killer Smartphone by Xiaomi   
1                                vry small size mobile   
2         Full display not working in all application.   
3                                      Value for Money   
4                              Not worth for the money   
..                                                 ...   
275  Cemera quality,face unlock most important in t...   
276                                   Mi is best phone   
277                                     Its a OK Phone   
278                                              Redmi   
279                           Not worth for the price.   

                Customer name              Rating                  Date  \
0           Rishikumar Thakur  4.0 out of 5 stars     on 1 October 2018   
1                     Raza ji  3.0 out of 5 stars  on 15 September 2018   
2               Vaibhav Patel  3.0 out of 5 stars  on 18 September 2018   
3  

In [18]:
df.drop_duplicates

<bound method DataFrame.drop_duplicates of                                           Review Title  \
0         Another Midrange killer Smartphone by Xiaomi   
1                                vry small size mobile   
2         Full display not working in all application.   
3                                      Value for Money   
4                              Not worth for the money   
..                                                 ...   
275  Cemera quality,face unlock most important in t...   
276                                   Mi is best phone   
277                                     Its a OK Phone   
278                                              Redmi   
279                           Not worth for the price.   

                Customer name              Rating                  Date  \
0           Rishikumar Thakur  4.0 out of 5 stars     on 1 October 2018   
1                     Raza ji  3.0 out of 5 stars  on 15 September 2018   
2               Vaibhav Patel  3.0 

In [19]:
#Sentiment_for_each_comment
def CommentReview(comments):
    completions=openai.Completion.create(
        engine=deployment_name,
        prompt=(f"what type of comment - {comments} it is negative, positive or neutral?"),
        max_tokens=20
    )
    message=completions['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return message

cols = ['Category', 'Comments', 'ReviewType']
df2 = pd.DataFrame(columns=cols)
Sentiment_for_each_comment = pd.DataFrame(columns=cols)

for ind in df.index:
    cat = df['Category'][ind]
    com = df['Comments'][ind]
    Sentiment_for_each_comment = Sentiment_for_each_comment.append({'Category': df['Category'][ind], 'Comments':df['Comments'][ind], 'ReviewType': CommentReview(df['Comments'][ind])},ignore_index=True)


C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\3247127280.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Sentiment_for_each_comment = Sentiment_for_each_comment.append({'Category': df['Category'][ind], 'Comments':df['Comments'][ind], 'ReviewType': CommentReview(df['Comments'][ind])},ignore_index=True)
C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\3247127280.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Sentiment_for_each_comment = Sentiment_for_each_comment.append({'Category': df['Category'][ind], 'Comments':df['Comments'][ind], 'ReviewType': CommentReview(df['Comments'][ind])},ignore_index=True)
C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\3247127280.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

In [20]:
df_grouped = df.groupby("Category")["Comments"].apply(" ".join).reset_index()
df_grouped = df_grouped.rename(columns={"Comments": "Aggregated_Comments"})
df_grouped

,Category,Aggregated_Comments
0,Battery,"Thanks alot Amazon,ecom & seller.perfect produ..."
1,Camera,Front camera is poor rest things are good Fron...
2,Delivery,Nice product. Delivery guy also. Nice product....
3,Display,Another Midrange killer Smartphone by Xiaomi\n...
4,Others,All ok but vry small size mobile Quite good wo...


In [21]:
#category wise summariszation of comments

nlp = spacy.load("en_core_web_sm")

def text_to_chunks(text):
  chunks = [[]]
  chunk_total_words = 0

  sentences = nlp(text)

  for sentence in sentences.sents:
    chunk_total_words += len(sentence.text.split(" "))

    if chunk_total_words > 1000:
      chunks.append([])
      chunk_total_words = len(sentence.text.split(" "))

    chunks[len(chunks)-1].append(sentence.text)
  
  return chunks

def Comment_category(Comments):
    completions=openai.Completion.create(
        engine=deployment_name,
        prompt=(f"Give Summariaztion for the following comments:{Comments}"),
        max_tokens=200
    )
    message=completions['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return message

print("Category wise Summarization of comments:")

cols = ['Category','Summarized Comment']
summariszation_of_comments = pd.DataFrame(columns=cols)

for ind in df_grouped.index:
    cat = df_grouped['Category'][ind]
    com = df_grouped['Aggregated_Comments'][ind]
    chunks = text_to_chunks(com)
    for chunk in chunks:
      chunk_summaries = []
      chunk_summary = Comment_category(" ".join(chunk))
      chunk_summaries.append(chunk_summary)
    summary = " ".join(chunk_summaries)
    print(df_grouped['Category'][ind], ":"+"\n"+Comment_category(summary), end="\n")
    summariszation_of_comments = summariszation_of_comments.append({'Category': df['Category'][ind], 'Summarized Comment': Comment_category(summary)},ignore_index=True)

Category wise Summarization of comments:
Battery :
The Xiaomi Mi 6 Pro offers an attractive, light design, an impressive camera and good sound quality. However, it has faced some technical issues, such as charging and lagging and it has taken away features such as the USB cable and front flash compared to older Mi models, which has made some customers feel disappointed as it is priced at 11,000 (3GB/32GB).


C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\3402581136.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summariszation_of_comments = summariszation_of_comments.append({'Category': df['Category'][ind], 'Summarized Comment': Comment_category(summary)},ignore_index=True)


Camera :
The Mi Redmi 6 Pro has a bad front camera, a decent rear camera and is heavy in weight. Additionally, its speakers give out a distorted sound which makes listening unpleasant. Despite having good fingerprint and face unlocking, its not suitable for selfies due to its low quality front camera. There are better options available at the same price range, suggesting that users should not buy the phone solely due to its low price tag.


C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\3402581136.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summariszation_of_comments = summariszation_of_comments.append({'Category': df['Category'][ind], 'Summarized Comment': Comment_category(summary)},ignore_index=True)


Delivery :
This customer was pleased with their purchase of a powerful mobile device from Amazon and its speedy delivery, but was unhappy with the delayed delivery of the charger.


C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\3402581136.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summariszation_of_comments = summariszation_of_comments.append({'Category': df['Category'][ind], 'Summarized Comment': Comment_category(summary)},ignore_index=True)


Display :
This product has great features, including a good display, fast processing, and a large battery, but its camera is only good, not special, due to the lack of a dual-camera setup.


C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\3402581136.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summariszation_of_comments = summariszation_of_comments.append({'Category': df['Category'][ind], 'Summarized Comment': Comment_category(summary)},ignore_index=True)


Others :
This customer had multiple issues with their Lenovo phone, including mobile data, WiFi, hot-spot, and Bluetooth not working as expected. To help with these problems, they purchased a protective, transparent back cover which they were pleased with. The customer was not pleased with the phone's screen size and found the phone to be quite thick. The customer did not recommend buying the phone, due to the issues with heat, and instead suggested looking to the Realme 2 for better value for money.


C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\3402581136.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summariszation_of_comments = summariszation_of_comments.append({'Category': df['Category'][ind], 'Summarized Comment': Comment_category(summary)},ignore_index=True)


In [22]:
file_name = 'output.xlsx'
# Load the workbook
writer = pd.ExcelWriter('output.xlsx', engine='openpyxl') 
workbook = openpyxl.Workbook()
writer.book = workbook

# Write the DataFrame to a new sheet
Sentiment_for_each_comment.to_excel(writer, index=False, sheet_name='Sentiment for each comment')
summariszation_of_comments.to_excel(writer, index=False, sheet_name='summariszation of comments')

# Save the workbook
writer.save()
print('DataFrame is written to Excel File successfully.')

C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\2252963973.py:5: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = workbook
C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\2252963973.py:12: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


DataFrame is written to Excel File successfully.
